In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms

Deep learning models run a lot faster on GPUs than on CPUs

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
data_transform = transforms.Compose([transforms.Resize((120, 120)),
                                     transforms.ColorJitter(0.05),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.RandomVerticalFlip(), 
                                     transforms.RandomRotation(20),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

In [4]:
images_dir = "../input/cell-images-for-detecting-malaria/cell_images/cell_images"
dataset = datasets.ImageFolder(images_dir, transform=data_transform)

In [5]:
test_size = int(0.2*len(dataset))
train_size = len(dataset) - test_size

train, test = random_split(dataset, lengths = [train_size, test_size])

print(f'Dataset size: {len(dataset)} | Train size: {len(train)} | Test size: {len(test)}')

train_loader = DataLoader(train, batch_size=32)
test_loader = DataLoader(test, batch_size=32)

Dataset size: 27558 | Train size: 22047 | Test size: 5511


In [6]:
classes=['infected','uninfected']

In [8]:
class Malariamodel(nn.Module):
    
    def __init__(self):
        super(Malariamodel, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
            
        self.fc1 = nn.Linear(64*15*15, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 2)
        self.drop = nn.Dropout2d(0.2)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)    # flatten out a input for Dense Layer
        out = self.fc1(out)
        out = F.relu(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.drop(out)
        out = self.fc3(out)
        
        return out

In [9]:
model = Malariamodel()
model.to(device)
error = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)


MosquitoNet(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=14400, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=12

In [10]:
num_epochs = 20
batch_size = 100 

for epoch in range(num_epochs):
    train_loss = 0.
    model.train()    # explictily stating the training
    
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        train = images.view(-1, 3, 120, 120)
        outputs = model(train)
        
        optimizer.zero_grad()
        loss = error(outputs, labels)
        loss.backward()    #back-propagation
        optimizer.step()
        
        train_loss += loss.item() * batch_size
     
    print("Epoch: {}, Loss: {:.4f}".format(epoch + 1, train_loss / len(train_loader.dataset)))

Epoch: 1, Loss: 1.1394
Epoch: 2, Loss: 0.5372
Epoch: 3, Loss: 0.4837
Epoch: 4, Loss: 0.4665
Epoch: 5, Loss: 0.4383
Epoch: 6, Loss: 0.4136
Epoch: 7, Loss: 0.4157
Epoch: 8, Loss: 0.3925
Epoch: 9, Loss: 0.3898
Epoch: 10, Loss: 0.3864
Epoch: 11, Loss: 0.3749
Epoch: 12, Loss: 0.3715
Epoch: 13, Loss: 0.3593
Epoch: 14, Loss: 0.3512
Epoch: 15, Loss: 0.3572
Epoch: 16, Loss: 0.3364
Epoch: 17, Loss: 0.3383
Epoch: 18, Loss: 0.3308
Epoch: 19, Loss: 0.3255
Epoch: 20, Loss: 0.3212


In [11]:
torch.save(model.state_dict(), "model.pth")